In [7]:
import pandas as pd
import datasets
from collections import Counter
import ast
import json

all_human_annotations = pd.read_csv('HUMAN ANNOTATIONS')

In [8]:
all_human_annotations


,review_point,paper_id,venue,focused_review,actionability,actionability_label,actionability_label_type,batch,grounding_specificity,grounding_specificity_label,...,helpfulness_label_type,professional_tone,professional_tone_label,professional_tone_label_type,valid_point,valid_point_label,valid_point_label_type,addressed_to_author,addressed_to_author_label,addressed_to_author_label_type
0,- There were too many missing details (for exa...,ARR_2022_60_review,ARR_2022,- Underdefined and conflation of concepts - Se...,"{'annotators': ['boda'], 'labels': ['4']}",NaN,NaN,1,"{'annotators': ['boda'], 'labels': ['3']}",NaN,...,NaN,"{'annotators': ['boda'], 'labels': ['1']}",NaN,NaN,"{'annotators': ['boda'], 'labels': ['0']}",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN
1,"- Section 4.3, model parameters and training. ...",ACL_2017_333_review,ACL_2017,There are some few details on the implementati...,"{'annotators': ['boda'], 'labels': ['5']}",NaN,NaN,1,"{'annotators': ['boda'], 'labels': ['5']}",NaN,...,NaN,"{'annotators': ['boda'], 'labels': ['1']}",NaN,NaN,"{'annotators': ['boda'], 'labels': ['0']}",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN
2,- The differences in results in Table 2 are ve...,ARR_2022_269_review,ARR_2022,- It is not clear for me about the novelty of ...,"{'annotators': ['boda'], 'labels': ['1']}",NaN,NaN,1,"{'annotators': ['boda'], 'labels': ['5']}",NaN,...,NaN,"{'annotators': ['boda'], 'labels': ['1']}",NaN,NaN,"{'annotators': ['boda'], 'labels': ['0']}",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN
3,"- While the language has been improved, there ...",ARR_2022_331_review,ARR_2022,"- While the language has been improved, there ...","{'annotators': ['boda'], 'labels': ['3']}",NaN,NaN,1,"{'annotators': ['boda'], 'labels': ['1']}",NaN,...,NaN,"{'annotators': ['boda'], 'labels': ['1']}",NaN,NaN,"{'annotators': ['boda'], 'labels': ['0']}",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN
4,1) The character tri-gram LSTM seems a little ...,ACL_2017_477_review,ACL_2017,1) The character tri-gram LSTM seems a little ...,"{'annotators': ['boda'], 'labels': ['5']}",NaN,NaN,1,"{'annotators': ['boda'], 'labels': ['4']}",NaN,...,NaN,"{'annotators': ['boda'], 'labels': ['1']}",NaN,NaN,"{'annotators': ['boda'], 'labels': ['0']}",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,* Originality The most closely related work of...,NIPS_2017_382,NIPS_2017,weakness that there is much tuning and other s...,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN,10,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,...,NaN,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN
1531,* Including a comparison to one of the methods...,bWXIut4pNM,EMNLP_2023,There are 3 potential changes that would impro...,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN,10,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,...,NaN,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN
1532,1. The time complexity of the learning algorit...,NIPS_2016_95,NIPS_2016,1. The time complexity of the learning algorit...,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN,10,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,...,NaN,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN
1533,"4) for the third point of definition one, is t...",NIPS_2016_537,NIPS_2016,weakness of the paper is the lack of clarity i...,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,NaN,10,"{'annotators': ['boda', 'cYVX6CPX', 'qhBkNx6n'...",NaN,...,NaN,"{'annotators': [

In [9]:
aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness', "professional_tone", 'valid_point', 'addressed_to_author']
processed_data = []

for index, row in all_human_annotations.iterrows():
    for aspect in aspects:
        ## get the most common label
        ## load the labels as json
        labels = row[aspect].replace("'", "\"")
        labels = json.loads(labels)['labels']
        if labels:
            # labels = ast.literal_eval(labels)
            frequency = Counter(labels)
            most_frequent_value, occurrence = frequency.most_common(1)[0]
        else:
            most_frequent_value = 'None'
            occurrence = 0
        if occurrence > 1:
            row[f'{aspect}_label'] = most_frequent_value
            if occurrence > 2:
                row[f'{aspect}_label_type'] = 'gold'
            else:
                row[f'{aspect}_label_type'] = 'silver'
        else:
            row[f'{aspect}_label'] = 'None'
            row[f'{aspect}_label_type'] = 'None'
    processed_data.append(row)

processed_data = pd.DataFrame(processed_data)



In [10]:
processed_data.to_csv('all_human_annotations_processed.csv', index=False)

In [11]:
print(len(processed_data))

1535


In [12]:
import krippendorff
import numpy as np
## group entries by each batch number, and then output the statiscs for each aspect also write it to txt file
with open('annotation_statistics.txt', 'w') as f:
    for batch in range(1, 11):
        print(f'Batch: {batch}')
        f.write(f'Batch: {batch}\n')
        batch_data = processed_data[processed_data['batch'] == batch]
        print(len(batch_data))
        f.write(f'Number of entries: {len(batch_data)}\n')
        for aspect in aspects:
            print(f'Aspect: {aspect}')
            f.write(f'Aspect: {aspect}\n')
            print(batch_data[f'{aspect}_label_type'].value_counts())
            f.write(f'{batch_data[f"{aspect}_label_type"].value_counts()}\n')
            print('---------------------------------')
            f.write('---------------------------------\n')

    ######### total statistics
    print('Total Statistics')
    f.write('Total Statistics\n')
    print(len(processed_data))
    f.write(f'Number of entries: {len(processed_data)}\n')
    for aspect in aspects:
        print(f'Aspect: {aspect}')
        f.write(f'Aspect: {aspect}\n')
        print(processed_data[f'{aspect}_label_type'].value_counts())
        f.write(f'{processed_data[f"{aspect}_label_type"].value_counts()}\n')
        print('---------------------------------')
        f.write('---------------------------------\n')


Batch: 1
25
Aspect: actionability
actionability_label_type
None    25
Name: count, dtype: int64
---------------------------------
Aspect: grounding_specificity
grounding_specificity_label_type
None    25
Name: count, dtype: int64
---------------------------------
Aspect: verifiability
verifiability_label_type
None    25
Name: count, dtype: int64
---------------------------------
Aspect: helpfulness
helpfulness_label_type
None    25
Name: count, dtype: int64
---------------------------------
Aspect: professional_tone
professional_tone_label_type
None    25
Name: count, dtype: int64
---------------------------------
Aspect: valid_point
valid_point_label_type
None    25
Name: count, dtype: int64
---------------------------------
Aspect: addressed_to_author
addressed_to_author_label_type
None    25
Name: count, dtype: int64
---------------------------------
Batch: 2
50
Aspect: actionability
actionability_label_type
silver    28
None      12
gold      10
Name: count, dtype: int64
----------